<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
import api
import imp
import araalictl

In [ ]:
# if you are a api dev and make changes
imp.reload(araalictl)
imp.reload(api)

In [ ]:
api.g_tenant = None

# Runtime Assessment

In [ ]:
assess = api.Assess()

## Summary

In [ ]:
api.Table(assess.summary())

In [ ]:
api.Table(assess.top_consumed_services())

## Inactive Ports

In [ ]:
api.Table(assess.inactive_ports())

## Perimeter Egress - Orgs and Countries

In [ ]:
api.Table(assess.geo_org_accessors())

### Country

In [ ]:
links = _.links

In [ ]:
api.Table(api.value_count([a["geo_service_info"][0]["country"] for a in links]))

### Org

In [ ]:
api.Table(api.value_count([a["geo_service_info"][0].get("org", "") for a in links]))

## Important Lenses

In [ ]:
api.Table(assess.starred_lenses())

In [ ]:
api.Table(assess.databases())

In [ ]:
api.Table(assess.dbaas())

In [ ]:
api.Table(assess.important_processes())

# Get all lenses

In [ ]:
api.Table(api.Lens.get(enforced=False, starred=False, all=True),
          #lambda o: o["type"] == "fqdn"
         )

In [ ]:
lens = _.links

In [ ]:
api.Table(api.value_count([a.type() for a in lens]))

In [ ]:
api.Table(lens, lambda o: o["type"] == "app")

In [ ]:
api.Table(lens, lambda o: o["type"] == "ip")

In [ ]:
api.Table(lens, lambda o: o["type"] == "fqdn")

## Count them

In [ ]:
len(lens)

## Pick one of the links

In [ ]:
api.Table([a for a in lens if a.obj.get("app", "") == "kube-system"])

In [ ]:
api.Table([a for a in lens if a.obj.get("fqdn", "") == "github.com"])

In [ ]:
api.Table([a for a in lens if a.obj.get("ip", "") == "169.254.169.254"])

In [ ]:
mylens = _.links[0]

# Lens to App Policies or Links

In [ ]:
app = mylens.app()

In [ ]:
api.Table(app.stats())

In [ ]:
api.LinkTable(app.iterlinks())

# App to Template

In [ ]:
app.template()

## Store the template from output above into a variable

In [ ]:
t = _

## Indexed summary of the template

In [ ]:
api.Table(t.index)

## Nodes of the template - so we can do some editing

In [ ]:
api.Table(t.nodes())

## If you want to keep just one process/service and its links

In [ ]:
t.keep_node(0)

## View the locally modified template

In [ ]:
api.Table(t.nodes())

## If you want to delete some nodes

In [ ]:
t.delete_nodes(*range(3,4))

In [ ]:
api.Table(t.nodes())

## If you want to update some nodes

In [ ]:
t.update_node(2, {"dns_pattern": ".+", "dst_port": 10250})

In [ ]:
t.update_node(0, {"zone": ".+"})

## If you want to rename the node

In [ ]:
t.rename("any-kube-system-metrics-server")

## If you want to publish the template as a public one in Araali Marketplace

In [ ]:
t.public = True

## If you only want to save locally

In [ ]:
t.save()

## If you want to load from a locally saved template

In [ ]:
t = api.Template(fname="template-public-any-k8s-nodes-kubelet.yaml")

## If you want to push to Araali Marketplace

In [ ]:
t.push()

In [ ]:
t

# Load up links so you can measure the efficacy of templates

In [ ]:
import copy
api.Runtime.zone2apps = None
run = api.Runtime()
run.refresh()
runlinks = copy.deepcopy(list(run.iterlinks()))

## Summary of alerts

In [ ]:
api.Table(run.link_stats(all=True, only_new=True, runlink=runlinks))

# Load all public teamplates in Araali Marketplace

In [ ]:
matchers = api.Templates(public=True)

## Add a local template to it

In [ ]:
t = api.Template(fname="template-any-kube-system-metrics-server.yaml")

In [ ]:
matchers.add(t)

## View the list of templates that you will be matching with

In [ ]:
api.Table(matchers.templates)

# Run all the links through matchers with post filters

In [ ]:
api.LinkTable(matchers.run(runlinks, matched=False), 
          lambda o: o["type"] == "NAE",
          #lambda o: o["client"]["process"] == "chrome"
         )

## Analyze all the links that did not match

In [ ]:
links = _.links

In [ ]:
len(links)

## Take a link and make a template out of it

In [ ]:
links[0].template()

In [ ]:
t = _

In [ ]:
t.rename("zone-app-process")

In [ ]:
api.Table(t.index)

In [ ]:
api.Table(t.nodes())

In [ ]:
t.update_node(1, {"dns_pattern": ".+"})

In [ ]:
t.save()

In [ ]:
api.Table(api.value_count([a.type for a in links]))

In [ ]:
api.Table(links, lambda o: o["client"]["process"] == "metrics-server")

In [ ]:
api.Table(api.value_count([a.client.app for a in _.links]))

In [ ]:
api.LinkTable([a for a in links if a.server.dns_pattern is None])

In [ ]:
api.Table(api.value_count([a.server.subnet for a in links if hasattr(a.server, "subnet")]))

In [ ]:
api.Table(api.value_count([a.server.dns_pattern for a in links]))

In [ ]:
api.Table(api.value_count([a.client.process for a in links]))

In [ ]:
api.Table(api.value_count([a.type for a in links]))

# Score after all the template based work

In [ ]:
api.Table(run.link_stats(all=True, only_new=True, runlink=runlinks))

## Analyze efficacy of individual matchers

In [ ]:
api.LinkTable([a for a in runlinks if hasattr(a, "policy")])

In [ ]:
links = _.links

In [ ]:
len(links)

## Frequency count

In [ ]:
api.Table(api.value_count([a.policy for a in links]))